In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.sys.path.append("/home/izmailovpavel/Documents/Projects/pristine-ml")
os.sys.path.append("/home/izmailovpavel/Documents/Projects/mode-geometry/")
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import glob
from itertools import accumulate, starmap, islice
import pandas as pd
from copy import deepcopy
from mpmath import mp; mp.dps = 120; mp.prec =300
from oil.utils.utils import multiGen, cur
from oil.utils.utils import cosLr
from oil.extra.eigen2 import oja_grad,oja_grad2, SGHA_grad,SGHA_grad2, GradLoader
from oil.extra.eigen2 import SGD,SVRG, H_gen
#from oil.extra.mvm import flatten, unflatten_like, autoHvpBatch, autoFvpBatch

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from models import VGG16
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import tqdm
import data
import models
import utils
import understanding_utils
from understanding_utils import UnderstandingNetwork
from torch.autograd import Variable

In [8]:
path = "/home/izmailovpavel/datasets/cifar100/"
loaders, num_classes = data.loaders(
    "CIFAR10",
    "/home/izmailovpavel/datasets/",
    128,
    4,
    "VGG",
    True
)

Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


In [10]:
torch.backends.cudnn.benchmark = True
num_classes = 10
criterion = F.cross_entropy

In [12]:
architecture = getattr(models, "SmallConvFC")

model = architecture.base(num_classes=num_classes, **architecture.kwargs)
model.cuda()
checkpoint = torch.load("/home/izmailovpavel/Documents/ckpts/c10/smallconvfc/run1/checkpoint-200.pt")
model.load_state_dict(checkpoint['model_state'])
print(utils.test(loaders["test"], model, criterion))

{'nll': 0.4700113780975342, 'accuracy': 84.84, 'loss': 0.4700113780975342}


In [14]:
total = 0
for param in model.parameters():
    total += param.numel()
total

134122

In [85]:
# def vec_to_model(vec):
#     new_model = architecture.base(num_classes=num_classes, **architecture.kwargs).cuda()
#     prev_i = 0
#     for param in new_model.parameters():
#         param.data = vec[prev_i:prev_i+param.numel()].reshape(param.shape).data
#         prev_i += param.numel()
#     return new_model
        
# def model_to_vec(model):
#     vec = []
#     for param in model.parameters():
#         vec.append(param.reshape(-1))
#     return torch.cat(vec)

def flatten_params(param_lst):
    return torch.cat([param.reshape(-1) for param in param_lst])

In [58]:
param_vec = Variable(torch.zeros(134122), requires_grad=True)
param_vec.data = model_to_vec(model)

In [66]:
new_model = vec_to_model(param_vec)

In [67]:
utils.test(loaders["test"], new_model, criterion)

{'accuracy': 84.84, 'loss': 0.4700113780975342, 'nll': 0.4700113780975342}

In [240]:
def Hessian():
    eps = 1e-5
    for i, (x, y) in enumerate(loaders["train"]):
        model.zero_grad()
        x = Variable(x).cuda()
        y = Variable(y).cuda()
        batch_size = x.shape[0]
        output = model(x)
        loss = criterion(output, y)
        def HessVec(v):
            grad_f = torch.autograd.grad(loss, model.parameters(), create_graph=True)#, retain_graph=True)
            grad_f = flatten_params(grad_f)
            z = grad_f @ v
            z.backward(retain_graph=False)
            return flatten_params([param.grad / batch_size for param in model.parameters()])
        if i > 5:
            break
        yield [HessVec]

In [244]:
# def FullHessVec(vec):
#     ans = torch.zeros_like(vec)
#     for Hv in Hessian():
#         ans += Hv(vec)
#     return ans

In [257]:
param_vec = Variable(torch.randn(134122), requires_grad=True).cuda()
param_vec.data /= torch.norm(param_vec)
param_vec2 = Variable(torch.randn(134122), requires_grad=True).cuda()
param_vec2.data /= torch.norm(param_vec2)
for Hv in Hessian():
#     break
#     
#     param_vec.data *= 2
    print(Hv[0](param_vec).shape)
for Hv in Hessian():
    print(Hv[0](param_vec2).shape)

torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])
torch.Size([134122])


In [252]:
grads = GradLoader(oja_grad, multiGen(Hessian))

In [253]:
lrs = []
ws = []

log = lambda w,lrr,_: (lrs.append(lrr),ws.append(w))

In [254]:
w0 = Variable(torch.randn(134122), requires_grad=True).cuda()
w0 /= torch.norm(w0)
w = Variable(torch.randn(134122), requires_grad=True).cuda()

lr = lambda e: .3*.05#*2e-1*cosLr(num_epochs)(e)
num_epochs = 10

w = SVRG(grads,w0,w,lr,num_epochs,log)

sas


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation

In [ ]:
def ojas_grad(Aw, w):
    return -(Aw - (w@Aw)*w)

def SGD(matvec, w, grad_fun, lr, num_epochs):
    for epoch in range(num_epochs):
        for grad in grad_multigen:
            w = w - lr(epoch)*grad(w)
            log(w, lr(epoch))
    return w